In [1]:
from dotenv import load_dotenv
import json
import os
import pandas as pd
from pyoso import Client

load_dotenv()
OSO_API_KEY = os.environ['OSO_API_KEY']
client = Client(api_key=OSO_API_KEY)

In [2]:
MEASUREMENT_PERIODS = {
    'M1': '2025-04-05', # Feb
    'M2': '2025-05-02', # Mar
    'M3': '2025-05-21', # Apr
    'M4': '2025-06-09', # May
}

In [3]:
OSS_FUNDING_PATH = '../../../../../../../GitHub/oss-funding/data/funders/optimism/uploads/retrofunding_s7.csv'
RETRO_FUNDING_PATH = '../../../../../../../GitHub/Retro-Funding/results/S7'

In [4]:
rewards = []
for mp in MEASUREMENT_PERIODS.keys():
    _path = f"{RETRO_FUNDING_PATH}/{mp}/outputs/{mp}_consolidated_rewards.csv"
    _df = pd.read_csv(_path)
    _df['round_type'] = _df['round_id'].replace({7: 'Devooling', 8: 'Onchain Builders'})
    _df['period'] = mp
    rewards.append(_df.drop(columns=['filename', 'round_id']))
    
df_rewards = pd.concat(rewards, axis=0, ignore_index=True)
df_rewards

,op_atlas_id,op_reward,round_type,period
0,0x0008577196fa6ec286440b418e2f6305fc10e62ce759...,4408.20,Devooling,M1
1,0x0008577196fa6ec286440b418e2f6305fc10e62ce759...,1367.00,Onchain Builders,M1
2,0x000c2ce4773defb3010a58d3800d0ec9d432189c574b...,7718.74,Onchain Builders,M1
3,0x02065e72fe4eebfa1ebca19238e6147c8571c2b7fefa...,576.41,Onchain Builders,M1
4,0x0674e95c746371103249c2c987d64515a4ed5c19dc3f...,0.00,Onchain Builders,M1
...,...,...,...,...
1595,0xfe8e35b3487bd0e0457b9431b12b73403ca1f5c9c19d...,982.18,Onchain Builders,M4
1596,0xfeafe16f8c1a25aebe754f3653340a17943a483a3e38...,833.41,Devooling,M4
1597,0xff1edf67227651293a9cff4dce3577c6ca52bf8c7c6f...,2390.03,Onchain Builders,M4
1598,0xff350d81524a79a1062f5f4977b45406b822fb4f150b...,0.00,Onchain Builders,M4


In [5]:
df_projects = client.to_pandas("""

WITH
  projects AS (
    /* your original 528 projects */
    SELECT DISTINCT
      pbc.project_id,
      pbc.project_name,
      p.display_name,
      CASE
        WHEN pbc.collection_name IN ('8-1','8-2','8-3','8-4')
          THEN 'Onchain Builders'
        ELSE 'Devtooling'
      END AS round_type
    FROM projects_by_collection_v1 AS pbc
    JOIN projects_v1 AS p
      ON pbc.project_id = p.project_id
    WHERE pbc.collection_source = 'OP_ATLAS'
  ),

  atlas_filtered AS (
    /* only those ATLAS artifacts you care about */
    SELECT
      a.project_id,
      a.artifact_id
    FROM int_artifacts_by_project_in_op_atlas AS a
    JOIN projects AS p
      ON a.project_id = p.project_id
     AND (
         (a.artifact_type = 'REPOSITORY' AND p.round_type = 'Devtooling')
      OR (a.artifact_type IN ('DEPLOYER', 'DEFILLAMA_PROTOCOL', 'SOCIAL_HANDLE') AND p.round_type = 'Onchain Builders')
     )
  ),

  atlas_ossd AS (
    /* all distinct project → OSSD‐project links */
    SELECT DISTINCT
      af.project_id,
      o.project_id AS ossd_project_id
    FROM atlas_filtered AS af
    JOIN int_artifacts_by_project_in_ossd AS o
      ON o.artifact_id = af.artifact_id
  )

SELECT
  p.project_id,
  p.project_name AS op_atlas_id,
  p.display_name,
  p.round_type,

  /* if there was a match, show it; otherwise UNKNOWN */
  COALESCE(p2.project_name, 'UNKNOWN') AS ossd_filename

FROM projects AS p

/* bring in zero‐to‐many ossd matches */
LEFT JOIN atlas_ossd AS ao
  ON ao.project_id = p.project_id

/* look up the project name in your OSSd table */
LEFT JOIN projects_v1 AS p2
  ON p2.project_id = ao.ossd_project_id

ORDER BY
  p.project_id,
  ossd_filename

""")

MAPPED_DUPES = {
    '0x9b639cc6061e41a1d12adb39619a2fddb7005074cf965a3a6f4eabc6eea31112': 'wighawag',
    '0xebe03c3d6d33cad60124b9b05ef6e2ff056293a1de3c5fa51dfbb90c86c14bf7': 'web3py-ethereum',
    '0x5cfdb8c453a7eede1946071e9d7417d01cc2116f57563b87dc0094a6bdb9a081': 'aspiringdevelopers',
    '0x92f33d0611c514cafbf54780ec7925a6080a37a192683444c30116cb099d95e7': 'dspytdao',
    '0xb926ab918147d2264dba94a429eb07986aa533fd6062a78736d3eda4e0b4a69b': 'gfx-labs',
    '0x4602ef8ac6aa7a0e04813c9ae6474f0836746b12054e492ad75d688e8521b494': 'UNKNOWN',
    '0x27f345fdead33d831d6022462628b6a9ad384e7681ee58648824d17c4addc089': 'UNKNOWN',
    '0xa2aee09bb6421f6d4c992822a7dcc110527f91486ee9d9dbf7d5af0a148b41f7': 'UNKNOWN',
    '0xef71b036123a72aa9aa64afb1263751bdfc7e7e4e63ec658c323136dc3a88d37': 'UNKNOWN',
    '0x12da4f117ab1a57f2f02078df3dbb9ecd4c66fe7b40535314b21218529f554cd': 'UNKNOWN',
    '0xaa1b878800206da24ee7297fb202ef98a6af0fb3ec298a65ba6b675cb4f4144b': 'UNKNOWN',
    '0xeb6d215732b1ed881e718faa8bf8b4b88a94edf021efa9a3cf3e2cc3c22b0961': 'UNKNOWN',
    
}
UNMAPPED = {
  "0x6fc25d7c763441933c53333f0f9105b37cc74d88f21ef61b3b7eaf94cba45bfb": "amped-finance",
  "0x800707befd6c94e7cea794200a6fee1155e54ca94bc10ee85d15c3f23845b10c": "ethereumfollowprotocol",
  "0x02065e72fe4eebfa1ebca19238e6147c8571c2b7fefae4e86e62ce768edceeb9": "exactly",
  "0xc8a6f84677aca1ce288713e63616048e307d4c2d50aa76d65bc46e4c75aa3d53": "flatmoney",
  "0x259b729af86ba38c0c84be9b739dfbca109a6df985371d8cabd43116d18d7a1a": "flayerlabs",
  "0x8fbddc02ddbc9c9b303b7bbb645aa933541876a31a93d7dea165fb0283d7f685": "holdstation",
  "0x0d72f0fd9ca1d475f2fc199ebe456f80212d24d02d9dbabca6137442e61c5f3f": "infecteddotfun",
  "0xf9513b5bfca3be3513450bd56b48ae485ce157aa56c1d8b2e06a68bce0b8d084": "javsphere",
  "0x2bdfa034cf1e853db8ed4b30531270c7a62d66550cd23383819e816abb234f0a": "kids-table-inc",
  "0xe6aa409da818b166ea4d354534d92745f2153fca126c1c38de675d42c3c559fe": "mintrich",
  "0xb26d9315cec30e60b52efd523c25a260c41d573458b246e60cf2de76eefd15a4": "mintswapfinance",
  "0x469f813ce1b0c64293d1f884363808f076d27abc8c6060bd3abe46bedd0d7982": "nepfinance",
  "0xe6371e0b190ff5db00617519599cbbafe719db0386e42a0af106b8c7f9161b63": "nerzofc",
  "0x40ee6048c99aefd288f6133db207849d0951fe6b62afa1cca0db5a4ad7f558de": "party-dao",
  "0x9fb539aa3ddb63cda1c29b50fbeaa6071f290926ce10f3d224233577b823454e": "party-dao",
  "0xb620f012eb197e5661f65c1a5aee6a4bbb43629f0056879ae4dc66adf8383ae3": "party-dao",
  "0x7d5bcf0b123ca5611f1bb0c889816753cc215b0a230849c8464c6c2a08c4eddc": "party-dao",
  "0x7eb6ba7a1057659d04ec414446c1b32509bf57704958139266c7120903446ad6": "pinto-org",
  "0xe329147afe5f04d45a1ba005819ef75b36136576aff042b0714ba793f0727938": "poopzter",
  "0x867b0364ac86ff4cc61a47ee99a0f96c8b952af8728442a048f7e3ac5892aae9": "poppunk-llc",
  "0x85bef37b528ec9d7e1a9cb864e298e3c117071507acd3f522c28561435d167f5": "rhinofi",
  "0x1f60db63651f74e2fc8b3ff900d05142552a763c31caf3a8c8e62b0953f02bf2": "stvol-official",
  "0xa09305aa57a60e362b11911666f6310f46c959ead9116333d2e7ca91aee180bf": "superaccount-superchaineco",
  "0x31704fee34d3edec158b5df714b613642bfbbb58ad7466204ded66a3f811b979": "superchain-safe-protofire", 
  "0x7f0e98c70d1401fb100dc804f28743bb567a2fee12315f52300113df514b7bd3": "superfluid",
  "0x0b8acaf73a25ba1f64bf829f9678beb8b7edf825353ebea38085590d78d74d5d": "unimon-alma-labs",
  "0x63d14cd07db0f56a91cb2f8972a29a225d24ea739034597f7c3e22f53817f7a8": "unimon-alma-labs",
  "0x40cda7edf27894d647d9433ded29290966f5c258dade2c3ea36427fccf0ef3c9": "veildotcash",
  "0x5831a5bc2b6e683bcb8e392453a85c5e7f755aa700e827ed9c3e1c6d67fb2a96": "worldrepublicorg"
}

df_projects['ossd_filename'] = df_projects.apply(lambda x: MAPPED_DUPES.get(x['op_atlas_id'], x['ossd_filename']), axis=1)
df_projects['ossd_filename'] = df_projects.apply(lambda x: UNMAPPED.get(x['op_atlas_id'], x['ossd_filename']), axis=1)

df_projects

,project_id,op_atlas_id,display_name,round_type,ossd_filename
0,+K3OrDKvASiVPDw6YwbzwsamF6URxSYCSlUun/KdTDU=,0x363b77a4022f98f4d529b8a771d22f49b1092293980a...,viem-tracer,Devtooling,viem-tracer-rubilmax
1,+ONsq1Xbp0mkKt4F9sl9X1hS/AaxD9MwE3tVjxMwZ9M=,0xf4f7d13f2ac3c9805998d8540950883b872d7c5d7c81...,evmc,Devtooling,evmc-joshstevens19
2,+PfbGQq2fXc65J+R0ni5deEPvSbqCGwIUSGfsnt3swM=,0x2bb095e1f297c71da8bd4ee15097abad3b99e299fe14...,solidity-coverage,Devtooling,solidity-coverage-sc-forks
3,+Xyt1tUxrXrbR9UH3h/0BmCBJJpBDBuFyJFBdmt/U44=,0x46d819ee72e9b51fb0175286e6ade010776b39cdb56a...,Team Finance,Onchain Builders,team-finance
4,+inHxf/47M81HpCJDOPqeN8ZQrT0nC0q1wfs9HzHSTI=,0x9f207ec89f5ddbcb96d2551d6633a0da7595b1070562...,Mykhailo Kashkarov,Onchain Builders,UNKNOWN
...,...,...,...,...,...
531,zCbuxTOy64SbO/+l0E7LtLS2VMhiw+1o0eedXw2s/ao=,0x991062ec81edb818a542e45d7c142cc62bc06e68b582...,Titan Layer,Onchain Builders,UNKNOWN
532,zVWBMiOJHTeEPXEgWSl1B+tsS8W9vwfv2QU32ANB8yY=,0xae3efd865f9584671ede5e94c83190b0aea5ac5d77eb...,Base Logos,Onchain Builders,UNKNOWN
533,zdNq7Cvjqx03oRiL22rCANVA7/8MBGaoc3InNwm3LCA=,0xb620f012eb197e5661f65c1a5aee6a4bbb43629f0056...,ORB,Onchain Builders,party-dao
534,zpXh+xI4A77C9Y/ortXA0jU/33Zj+QztUOpW3hJ+w8A=,0xe20f59eb1c8f9cb72e07b8d0f067bac997614fd6db06...,L2Pass,Onchain Builders,l2pass


In [6]:
rewarded_projects = list(df_rewards[df_rewards['op_reward'] > 0]['op_atlas_id'].unique())
len(rewarded_projects)

261

In [7]:
df_known_projects = df_projects[
  (df_projects['ossd_filename'] != 'UNKNOWN') 
& (df_projects['op_atlas_id'].isin(rewarded_projects))
]
dupes = df_projects.groupby('op_atlas_id')['ossd_filename'].nunique().sort_values()
dupes = list(dupes[dupes>1].index)
df_projects[df_projects['op_atlas_id'].isin(dupes)][['op_atlas_id', 'display_name', 'ossd_filename']].drop_duplicates()

,op_atlas_id,display_name,ossd_filename


In [8]:
dupes = df_projects.groupby('ossd_filename')['op_atlas_id'].nunique().sort_values()
dupes = list(dupes[dupes>1].index)
dupes = [x for x in dupes if x != 'UNKNOWN']

df_dupes = (
    df_projects[df_projects['ossd_filename'].isin(dupes)]
    [['op_atlas_id', 'display_name', 'ossd_filename']]
    .drop_duplicates().sort_values(by='ossd_filename')
)

#df_dupes

In [9]:
PROJECT_MAPPING = (
    df_projects[['op_atlas_id', 'ossd_filename']]
    .drop_duplicates()
    .set_index('op_atlas_id')['ossd_filename']
    .replace({'UNKNOWN': None})
    .sort_values()
    .to_dict()
)
PROJECT_NAMES = (
    df_projects[['op_atlas_id', 'display_name']]
    .drop_duplicates()
    .set_index('op_atlas_id')['display_name']
    .replace({'UNKNOWN': None})
    .sort_values()
    .to_dict()
)

In [10]:
df = df_rewards.copy()
df['to_project_name'] = df['op_atlas_id'].map(PROJECT_MAPPING)
df['funding_date'] = df['period'].map(MEASUREMENT_PERIODS)
df['application_name'] = df['op_atlas_id'].map(PROJECT_NAMES)
df['from_funder_name'] = 'optimism'
df['grant_pool_name'] = df['round_type'].apply(lambda x: f"retrofunding_s7_{x.lower().replace(' ','_')}")
df['metadata'] = df.apply(
    lambda x: json.dumps({
        'application_name': x['application_name'],
        'application_url': f"https://atlas.optimism.io/project/{x['op_atlas_id']}"        ,
        'token_amount': x['op_reward'],
        'token_unit': 'OP',
        'project_id': x['op_atlas_id']
    })
    , axis=1
)
df['amount'] = df['op_reward'] * 0.75
df = df[[
    'funding_date',
    'from_funder_name',
    'to_project_name',
    'amount',
    'grant_pool_name',
    'metadata'
]]
df

,funding_date,from_funder_name,to_project_name,amount,grant_pool_name,metadata
0,2025-04-05,optimism,createx,3306.1500,retrofunding_s7_devooling,"{""application_name"": ""CreateX"", ""application_u..."
1,2025-04-05,optimism,createx,1025.2500,retrofunding_s7_onchain_builders,"{""application_name"": ""CreateX"", ""application_u..."
2,2025-04-05,optimism,woonetwork,5789.0550,retrofunding_s7_onchain_builders,"{""application_name"": ""WOOFi"", ""application_url..."
3,2025-04-05,optimism,exactly,432.3075,retrofunding_s7_onchain_builders,"{""application_name"": ""Exa App"", ""application_u..."
4,2025-04-05,optimism,None,0.0000,retrofunding_s7_onchain_builders,"{""application_name"": ""SnakyCat"", ""application_..."
...,...,...,...,...,...,...
1595,2025-06-09,optimism,None,736.6350,retrofunding_s7_onchain_builders,"{""application_name"": ""Stoke Fire"", ""applicatio..."
1596,2025-06-09,optimism,foundry-storage-check-rubilmax,625.0575,retrofunding_s7_devooling,"{""application_name"": ""foundry-storage-check"", ..."
1597,2025-06-09,optimism,kelp-dao,1792.5225,retrofunding_s7_onchain_builders,"{""application_name"": ""Kelp"", ""application_url""..."
1598,2025-06-09,optimism,hermes-protocol,0.0000,retrofunding_s7_onchain_builders,"{""application_name"": ""Hermes"", ""application_ur..."


In [11]:
df.to_csv(OSS_FUNDING_PATH, index=False)

In [12]:
# for p in sorted(df['to_project_name'].dropna().unique()):
#     print(f"  - {p}")